# ロジスティック回帰を実装する

In [3]:
import tensorflow as tf
import time, shutil, os
from tensorflow.examples.tutorials.mnist import input_data

In [11]:
mnist = input_data.read_data_sets("data",one_hot=True)

# parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 100
display_step = 1

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Inferance
与えられたミニバッチに対してクラスの出力を返す

In [4]:
def inference(x):
    # 
    init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", [784,10], initializer = init)
    b  = tf.get_variable("b", [10], initializer = init)
    output = tf.nn.softmax(tf.matmul(x, W) + b)
    
    # 表示用
    tf.summary.histogram("weights",W)
    tf.summary.histogram("biases",b)
    tf.summary.histogram("output",output)

    return output

## Loss
損失関数（クロスエントロピー）

$loss = - \sum y_k log{o_k}$

reduce_sum : 与えられたリストの総和をとる

reduce_mean : 与えられたリストの平均を返す

In [9]:
def loss(output, y):
    elementwise_product = y * tf.log(output)
    
    xentropy = -tf.reduce_sum(elementwise_product, reduction_indices=1)
    
    loss = tf.reduce_mean(xentropy)
    
    return loss

## Training
パラメータの勾配を計算し，更新を行う

In [13]:
def training(cost, global_step):
    tf.summary.scalar("cost",cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    return train_op

## Evaluate


In [17]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("valisation error", (1.0 -accuracy))
    return accuracy

## Main
MNISTの手書き数字データを利用して分類を行う

In [23]:
if __name__ == "__main__":
    # logが存在しているなら削除
    if os.path.exists("logistic_logs"):
        shutil.rmtree("logistic_logs")
    
    with tf.Graph().as_default():
        x = tf.placeholder("float", [None,784]) # image = 28*28  = 784
        y = tf.placeholder("float", [None,10]) # 0~9 classes
        output = inference(x)
        cost = loss(output, y)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        train_op = training(cost, global_step)
        eval_op = evaluate(output, y)
        # merge graphs
        summary_op =  tf.summary.merge_all()
        # model save
        saver = tf.train.Saver()
        sess = tf.Session()
        # log write
        summary_writer = tf.summary.FileWriter(
            "logistic_logs",
            graph_def=sess.graph_def
        )
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        
        # training cycle
        for epoch in range(training_epochs):
            avg_cost = 0
            total_batch =  int(mnist.train.num_examples / batch_size)
            #
            for i in range(total_batch):
                minibatch_x,minibatch_y = mnist.train.next_batch(batch_size)
                # Fit
                sess.run(train_op, feed_dict={x: minibatch_x, y: minibatch_y})
                # Compute
                avg_cost += sess.run(
                    cost, feed_dict={x: minibatch_x,y:minibatch_y}) / total_batch
            # display logs
            if epoch % display_step == 0:
                print("Epoch: {:04d} cost{:.9f}".format(epoch+1, avg_cost))
                accuracy = sess.run(eval_op, feed_dict={x: mnist.validation.images,y :mnist.validation.labels})
                print("Validation Error: {}".format(1 - accuracy))
                summary_str = sess.run(summary_op, feed_dict={x:minibatch_x ,y:minibatch_y})
                summary_writer.add_summary(summary_str, sess.run(global_step))
                saver.save(sess, os.path.join("logistic_logs", "model-checkpoint"),global_step=global_step)
    
        print("optimizer finish")
        accuracy = sess.run(eval_op, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Test accuracy: {}".format(accuracy))

INFO:tensorflow:Summary name valisation error is illegal; using valisation_error instead.
Epoch: 0001 cost1.176746432
Validation Error: 0.14840000867843628
Epoch: 0002 cost0.662512755
Validation Error: 0.12879997491836548
Epoch: 0003 cost0.550707617
Validation Error: 0.12019997835159302
Epoch: 0004 cost0.496778782
Validation Error: 0.11320000886917114
Epoch: 0005 cost0.463786472
Validation Error: 0.10939997434616089
Epoch: 0006 cost0.440942136
Validation Error: 0.10699999332427979
Epoch: 0007 cost0.423933894
Validation Error: 0.10460001230239868
Epoch: 0008 cost0.410633938
Validation Error: 0.10199999809265137
Epoch: 0009 cost0.399885606
Validation Error: 0.10039997100830078
Epoch: 0010 cost0.390929992
Validation Error: 0.09939998388290405
Epoch: 0011 cost0.383318720
Validation Error: 0.0974000096321106
Epoch: 0012 cost0.376689271
Validation Error: 0.09600001573562622
Epoch: 0013 cost0.371023243
Validation Error: 0.09520000219345093
Epoch: 0014 cost0.365927096
Validation Error: 0.09219

In [24]:
!ls

data  logistic_logs  logistic_regression.ipynb	README.md  Untitled.ipynb


In [ ]:
!tensorboard --logdir=logistic_logs

TensorBoard 1.12.2 at http://likelihood:6006 (Press CTRL+C to quit)
